In [1]:
import multiprocessing
from multiprocessing import Pool

import pandas as pd
from PKM_multiprocess import ParallelCompute_PKM as PC_PKM

if __name__ == "__main__":
    pool = Pool(processes=11)
    Result = pool.map(PC_PKM, [-29, -25, -20, -15, -10, -5, 0, 5, 8])
    pool.close()
    pool.join()
    df = pd.DataFrame(Result)
    df = df
    nametable=[]
    # создаются новые таблицы после каждого расчета
    for i in range(len(Result)):
        nametable = "resdataTemp"+str(df['T_air'][i])+".xlsx"
#         print(nametable)
        dfgas_streams_zaryad=df['gas_streams_zaryad'][i]
        dfwater_streams_zaryad=df['water_streams_zaryad'][i]
        dfsyngas_streams_zaryad=df['syngas_streams_zaryad'][i]
        dfelectric_zaryad=df['electric_zaryad'][i]
        dfheaters_zaryad=df['heaters_zaryad'][i]
        dfaccumulation_zaryad=df['accumulation_zaryad'][i]
        dfgas_streams_razryad=df['gas_streams_razryad'][i]
        dfwater_streams_razryad=df['water_streams_razryad'][i]
        dfsyngas_streams_razryad=df['syngas_streams_razryad'][i]
        dfelectric_razryad=df['electric_razryad'][i]
        dfheaters_razryad=df['heaters_razryad'][i]
        dfaccumulation_razryad=df['accumulation_razryad'][i]
        
        dfelectric = dfelectric.T #Опционально, если неудобен такой вид таблицы -закоментить
        dfHeatex = dfHeatex.T #Опционально, если неудобен такой вид таблицы -закоментить
        print(nametable)
        print(dfelectric)
        print(dfHeatex)
        with pd.ExcelWriter(nametable) as writer:
            df.to_excel(writer, sheet_name="all_result", index=False)
            dfgas_streams_zaryad.to_excel(writer, sheet_name="gas_streams_zaryad", index=False)
            dfwater_streams_zaryad.to_excel(writer, sheet_name="water_streams_zaryad", index=False)
            dfsyngas_streams_zaryad.to_excel(writer, sheet_name="syngas_streams_zaryad", index=False)
            dfelectric_zaryad.to_excel(writer, sheet_name="electric_zaryad", index=False)
            dfheaters_zaryad.to_excel(writer, sheet_name="heaters_zaryad", index=False)
            dfaccumulation_zaryad.to_excel(writer, sheet_name="accumulation_zaryad", index=False)
            dfgas_streams_razryad.to_excel(writer, sheet_name="gas_streams_razryad", index=False)
            dfwater_streams_razryad.to_excel(writer, sheet_name="water_streams_razryad", index=False)
            dfsyngas_streams_razryad.to_excel(writer, sheet_name="syngas_streams_razryad", index=False)
            dfelectric_razryad.to_excel(writer, sheet_name="electric_razryad", index=False)
            dfheaters_razryad.to_excel(writer, sheet_name="heaters_razryad", index=False)
            dfaccumulation_razryad.to_excel(writer, sheet_name="accumulation_razryad", index=False)
    


SyntaxError: illegal target for annotation (1462795606.py, line 39)